In [1]:
import pdfplumber
import re
import json
import os
import PyPDF2


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [2]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [40]:
import json
from typing import List, Dict

def get_database_schema(driver) -> str:
    """Estrae automaticamente lo schema del database."""
    with driver.session() as session:
        # Query per ottenere i tipi di nodi
        node_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels
        RETURN COLLECT(labels) AS types
        """
        
        # Query per ottenere i tipi di relazioni e la loro struttura
        rel_query = """
        MATCH (a)-[r]->(b)
        WITH DISTINCT labels(a) AS sourceLabels, type(r) AS relType, labels(b) AS targetLabels
        RETURN sourceLabels, relType, targetLabels
        """
        
        # Esegui le query separatamente
        node_types = session.run(node_query).single()['types']
        relationships = list(session.run(rel_query))
        
        # Costruisci una descrizione dettagliata dello schema
        schema_description = "Schema del database:\n\nNodi:\n"
        for node_type in node_types:
            schema_description += f"- {':'.join(node_type)}\n"
        
        schema_description += "\nRelazioni e loro struttura:\n"
        for rel in relationships:
            source = ':'.join(rel['sourceLabels'])
            target = ':'.join(rel['targetLabels'])
            schema_description += f"- ({source})-[:{rel['relType']}]->({target})\n"
            
        return schema_description

def generate_cypher_query(llm, schema: str, question: str) -> str:
    """Genera una query Cypher basata sulla domanda dell'utente."""
    prompt = ChatPromptTemplate.from_template("""    
    Sei un esperto di Neo4j. Converti la seguente domanda in una query Cypher.
    
    {schema}
    
    Domanda: {question}
    
    Genera SOLO la query Cypher, senza backticks, senza la parola 'cypher' e senza commenti.
    La query deve usare ESATTAMENTE i nomi delle relazioni come appaiono nello schema.
    Assicurati di usare la proprietà 'nome' per fare match sui nomi di nodi.
    """)
    
    messages = prompt.format_messages(
        schema=schema,
        question=question
    )
    response = llm.invoke(messages)
    # Pulizia della risposta
    query = response.content.strip()
    query = query.replace('```cypher', '').replace('```', '').strip()
    return query

def execute_query(driver, query: str) -> List[Dict]:
    """Esegue la query Cypher e restituisce i risultati grezzi."""
    with driver.session() as session:
        result = session.run(query)
        print(f"Risultati grezzi della query: {result}")  # Mostra direttamente il risultato grezzo
        
        results = []
        for record in result:
            print(f"Record trovato: {record}")  # Mostra direttamente ogni record
            if 'p' in record:
                node = record['p']  # Estrai il nodo 'Piatto' dalla proprietà 'p'
                if node and 'nome' in node.keys():  # Verifica se la proprietà 'nome' esiste
                    results.append({'nome': node['nome']})  # Aggiungi il nome del piatto
                else:
                    results.append({'nome': 'Nome non disponibile'})
            else:
                results.append({'nome': 'Nodo Piatto non trovato'})
        return results

def interpret_results(llm, schema: str, question: str, results: List[Dict]) -> str:
    """Interpreta i risultati usando l'AI per generare una risposta naturale."""
    prompt = ChatPromptTemplate.from_template("""    
    Dai risultati di una query Neo4j, genera una risposta naturale e comprensibile alla domanda dell'utente.
    
    {schema}
    
    Domanda: {question}
    Risultati: {results}
    
    Fornisci una risposta completa ma concisa, evidenziando le informazioni più rilevanti.
    """)
    
    messages = prompt.format_messages(
        schema=schema,
        question=question,
        results=json.dumps(results, ensure_ascii=False, indent=2)
    )
    response = llm.invoke(messages)
    return response.content.strip()

def query_database(driver, llm, question: str) -> str:
    """Funzione principale che gestisce l'intero processo di query."""
    try:
        # Ottiene lo schema del database
        schema = get_database_schema(driver)
        print(f"Schema rilevato:\n{schema}")  # Debug
        
        # Genera la query Cypher
        cypher_query = generate_cypher_query(llm, schema, question)
        print(f"Query Cypher generata: {cypher_query}")  # Debug
        
        # Esegue la query
        results = execute_query(driver, cypher_query)
        
        # Stampa i risultati grezzi come parte del log
        print(f"Risultati grezzi (dati raw): {results}")  # Debug
        
        # Se non ci sono risultati
        if not results or all(result['nome'] == 'Nome non disponibile' for result in results):
            return "Non ho trovato risultati per questa domanda."
        
        # Interpreta i risultati
        risposta_naturale = interpret_results(llm, schema, question, results)
        
        # Restituisce la risposta naturale e i risultati grezzi
        return {
            "risposta_naturale": risposta_naturale,
            "risultati_grezzi": results  # Includiamo i risultati raw come sono
        }
        
    except Exception as e:
        return f"Si è verificato un errore durante l'elaborazione della tua domanda: {str(e)}"

# Esegui la domanda
domanda = "Che piatti che contengono sia Petali di Eco che Foglie di Mandragora posso mangiare se voglio che nessun ingrediente superi le quantitÃ  legali prescritte dal Codice di Galattico?"
risposta = query_database(driver, llm, domanda)

# Stampa la risposta
print(f"\nDomanda: {domanda}")
print(f"Risposta naturale: {risposta['risposta_naturale']}")
print(f"Risultati grezzi:\n{risposta['risultati_grezzi']}")


Schema rilevato:
Schema del database:

Nodi:
- Macrotecnica
- Piatto
- Tecnica
- Ordine
- Licenza
- Sostanza
- Pianeta
- Ristorante

Relazioni e loro struttura:
- (Piatto)-[:PREPARATO_CON_INGREDIENTE]->(Sostanza)
- (Piatto)-[:PREPARATO_CON_TECNICA]->(Tecnica)
- (Tecnica)-[:APPARTIENE_A]->(Macrotecnica)
- (Tecnica)-[:RICHIEDE_LICENZA]->(Licenza)
- (Pianeta)-[:DISTANZA]->(Pianeta)
- (Ristorante)-[:HA_LICENZA]->(Licenza)
- (Ristorante)-[:PREPARATO_DA]->(Piatto)
- (Ristorante)-[:PRESENTE_SU]->(Pianeta)

Query Cypher generata: MATCH (p:Piatto)-[:PREPARATO_CON_INGREDIENTE]->(:Sostanza {nome: "Petali di Eco"}),
      (p)-[:PREPARATO_CON_INGREDIENTE]->(:Sostanza {nome: "Foglie di Mandragora"}),
      (r:Ristorante)-[:PREPARATO_DA]->(p),
      (r)-[:HA_LICENZA]->(:Licenza {nome: "Codice di Galattico"})
RETURN p.nome
Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000001C2A8FE31D0>
Risultati grezzi (dati raw): []

Domanda: Che piatti che contengono sia Petali di Eco che

TypeError: string indices must be integers, not 'str'

In [ ]:
risulta

In [22]:
# Esempio di domanda
domanda = "Quali piatti preparati con la tecnica Grigliatura a Energia Stellare DiV?"
risposta = query_database(driver, llm, domanda)
print(f"\nDomanda: {domanda}")
print(f"Risposta: {risposta}")

Schema rilevato:
Schema del database:

Nodi:
- Macrotecnica
- Piatto
- Tecnica
- Ordine
- Licenza
- Sostanza
- Pianeta
- Ristorante

Relazioni e loro struttura:
- (Piatto)-[:PREPARATO_CON_INGREDIENTE]->(Sostanza)
- (Piatto)-[:PREPARATO_CON_TECNICA]->(Tecnica)
- (Tecnica)-[:APPARTIENE_A]->(Macrotecnica)
- (Tecnica)-[:RICHIEDE_LICENZA]->(Licenza)
- (Pianeta)-[:DISTANZA]->(Pianeta)
- (Ristorante)-[:HA_LICENZA]->(Licenza)
- (Ristorante)-[:PREPARATO_DA]->(Piatto)
- (Ristorante)-[:PRESENTE_SU]->(Pianeta)

Query Cypher generata: MATCH (p:Piatto)-[:PREPARATO_CON_TECNICA]->(t:Tecnica {nome: 'Grigliatura a Energia Stellare DiV'})
RETURN p.nome
Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000001C2A8F9A270>

Domanda: Quali piatti preparati con la tecnica Grigliatura a Energia Stellare DiV?
Risposta: Si è verificato un errore durante l'elaborazione della tua domanda: 'p'
